In [1]:
!pip install BeautifulSoup4

In [2]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
%pwd

'/home/souvic/mounted/btp/vahini/DataSets/CBSEData/Parser'

In [26]:

year="2010_1"
PATH = "/home/souvic/mounted/btp/ndata/aipmt/"+year+"/"
STORE = "/home/souvic/mounted/btp/ndata/extracted/OtherFiles/"+"2010_1_2"+"/"

In [27]:
%cd $PATH

/home/souvic/mounted/btp/ndata/aipmt/2010_1


In [28]:
import os

files = os.listdir(PATH)

In [29]:
def returnpick(x):
    with (open(x, "rb")) as openfile:
        return pickle.load(openfile)

In [ ]:
for file in files:
    fd = returnpick(file)
    rn = []
    count=0
    data= []
    for key in fd.keys():
        rn.append(key)
    
    for elem in rn:
        student = []
        count = count+1
        if count%10000 == 0:
            print(count)
        try:
            soup = BeautifulSoup(fd[elem],'html.parser')
            tab  = soup.find_all('table', align='center', width='600')
            if len(tab) < 2:
                continue
            info = tab[1].find_all('td', align='left')
            roll_no = info[0].text;
            name = info[1].text;
            father_name = info[2].text;
            category = info[3].text;
            sub_category = info[4].text;
            student = [roll_no, name, father_name, category, sub_category]
            data.append(student)
    #         break
        except:
            print(elem)


    df = pd.DataFrame(data, columns=['roll_no','name','father_name','category','sub_category'])
    df.to_csv(STORE+file[:-2]+".csv")


Add labels

In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf

In [13]:
def vocab():
    alpha = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
    special = [' ']
    vocab = alpha+special+['END']

    return vocab
    
def getIndex(df, maxlen, file):
    voc = vocab()
    char_index = dict((c, i) for i, c in enumerate(voc))
    
    X = []
    next_name = False
    
    # Club all the whitespaces
    
    trunc_X = [str(i).lower()[0:maxlen] for i in df['name']]
    mask = []
    for i in range(len(trunc_X)):
        name = trunc_X[i]
        tmp = []
        for char in str(name):
            if char not in voc:
                with open(file,'a') as f:
                    f.write(name+'\n')
                print(name)
                next_name = True
                break
            else:
                tmp.append(char_index[char])
        if next_name:
            mask.append(False)
            next_name = False
            continue
        mask.append(True)
        for k in range(0,maxlen - len(str(name))):
            tmp.append(char_index["END"])
        X.append(tmp)
            
    return np.asarray(X), df[mask]

In [14]:
import re
import numpy as np

def preprocess_df(df):
    df['name'] = [str(name).lower() for name in df['name']]
    df['name'] = [str(name).strip() for name in df['name']]
    df['name'] = [re.sub('[^a-zA-Z]+', ' ', str(name)) for name in df['name']]
    df['name'] = [re.sub('[ \t\n]+',' ', str(name)) for name in df['name']]
    df['name'].replace('', np.nan, inplace=True)
    df = df[df['name']!='nan']
    df.dropna(subset=['name'],inplace=True)
    return df

In [15]:
def returnGenderPred(df, model):
    test_x, df = getIndex(df, 30, 'oov.txt')
    pred = model.predict(np.asarray(test_x), verbose = 1)
    y_pred = np.argmax(pred, axis =1)
    df['Gender'] = y_pred
    return df

def returnCastePred(df, model):
    df = preprocess_df(df)
    test_x, df = getIndex(df, 30, 'oov.txt')
    pred = model.predict(np.asarray(test_x), verbose = 1)
    y_pred = np.argmax(pred, axis =1)
    df['Caste'] = y_pred
    return df

In [16]:
from keras.models import load_model


def get_gender_model():
    filename = "/home/souvic/mounted/btp/vahini/Models/ERTrain/SavedModels/FinalSet/CharLSTM/ERModel/ERModel"
    model = load_model(filename)
    return model

def get_caste_model():
    filename = "/home/souvic/mounted/btp/vahini/Name2Demographics/Models/AIEEEData/CNNModel/SavedModel/CasteModel"
    model = load_model(filename)
    return model

In [17]:
gender_model = get_gender_model()
caste_model = get_caste_model()

2022-02-14 00:59:38.734652: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-02-14 00:59:38.734725: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: obelix
2022-02-14 00:59:38.734737: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: obelix
2022-02-14 00:59:38.734873: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.86.0
2022-02-14 00:59:38.734913: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.86.0
2022-02-14 00:59:38.734923: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.86.0
2022-02-14 00:59:38.735385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in perf

In [18]:
print(STORE)

/home/souvic/mounted/btp/ndata/extracted/OtherFiles/2010_1/


In [19]:
files = os.listdir(STORE)

In [20]:
for file in files:
  df = pd.read_csv(STORE+file)
  df = preprocess_df(df)
  df = returnGenderPred(df, gender_model)
  df = returnCastePred(df, caste_model)
  del df['Unnamed: 0']
  df.to_csv(STORE+file)

12/12 [==============================] - 0s 7ms/step
